# Import

In [1]:
# standard libraries

import sys
import os
import numpy as np
import pandas as pd
from sklearn.metrics import mean_absolute_error
from sklearn.linear_model import LinearRegression
import matplotlib.pyplot as plt

# custom functions

sys.path.append("..")
from valicast.validation_methods import get_indices

# Utils

In [2]:
def get_X_y(df, n_lags, drop_init_lag=True):
  """Get X and y from univariate timeseries (df)"""

  df = df.interpolate().dropna()
  y = df.copy().iloc[:, 0]
  X = pd.concat([df.shift(lag).rename(lambda x: f"lag_{lag}", axis=1) for lag in range(1, n_lags+1)], axis=1)

  if drop_init_lag:
    return X.tail(len(X)-n_lags), y.tail(len(y)-n_lags)
  
  else:
    return X, y

# Init

In [3]:
project_dir = os.path.dirname(os.getcwd())

In [4]:
validation_methods = [
  "holdout",
  "inv_holdout",
  "rep_holdout",
  "cv",
  "cv_mod",
  "cv_bl",
  "cv_hvbl",
  "preq_bls",
  "preq_sld_bls",
  "preq_bls_gap",
  "preq_slide",
  "preq_grow"
]

In [5]:
validation_method2function = {
  "holdout": """get_indices(method="holdout", time_series_length=len(X)-len(id_prod), train_size=.8)""",
  "inv_holdout": """get_indices(method="inv_holdout", time_series_length=len(X)-len(id_prod), train_size=.8)""",
  "rep_holdout": """get_indices(method="rep_holdout", time_series_length=len(X)-len(id_prod), n_reps=5, train_size=.7, test_size=.2)""",
  "cv": """get_indices(method="cv", time_series_length=len(X)-len(id_prod), n_folds=5)""",
  "cv_mod": """get_indices(method="cv_mod", time_series_length=len(X)-len(id_prod), n_folds=5, gap_before=3, gap_after=3)""",
  "cv_bl": """get_indices(method="cv_bl", time_series_length=len(X)-len(id_prod), n_folds=5)""",
  "cv_hvbl": """get_indices(method="cv_hvbl", time_series_length=len(X)-len(id_prod), n_folds=5, gap_before=3, gap_after=3)""",
  "preq_bls": """get_indices(method="preq_bls", time_series_length=len(X)-len(id_prod), n_folds=5)""",
  "preq_sld_bls": """get_indices(method="preq_sld_bls", time_series_length=len(X)-len(id_prod), n_folds=5)""",
  "preq_bls_gap": """get_indices(method="preq_bls_gap", time_series_length=len(X)-len(id_prod), n_folds=5)""",
  "preq_slide": """get_indices(method="preq_slide", time_series_length=len(X)-len(id_prod), train_size=.8, n_reps=5)""",
  "preq_grow": """get_indices(method="preq_grow", time_series_length=len(X)-len(id_prod), train_size=.8, n_reps=5)"""
}

In [6]:
dataset_list = []

for dataset_id in [str(i).zfill(3) for i in range(1, 649)]:
  # I previously downloaded all the dataset locally through the R script download_data.R
  df = pd.read_csv(os.path.join(project_dir, "data", "datasets", f"dataset_{dataset_id}.csv"), index_col=0)
  
  # keep dataset only if univariate and more than 1,000 observations
  if df.shape[1] == 1 and df.shape[0] > 1000:
    dataset_list.append(str(dataset_id).zfill(3))

# Run experiment

In [7]:
results, results_row = pd.DataFrame(), 0

# 1. for each dataset...
for dataset_id in dataset_list:

  # get dataset
  X, y = get_X_y(pd.read_csv(os.path.join(project_dir, "data", "datasets", f"dataset_{dataset_id}.csv"), index_col=0), n_lags=50)
  
  # the latest 10% of observations will not be used neither for training nor for test
  id_prod = np.arange(start=int(len(X)*.9), stop=len(X), step=1, dtype=int)
  id_nonprod = np.arange(start=0, stop=int(len(X)*.9), step=1, dtype=int)  

  # 2. for each predictive model...
  for model_name in ["LinearRegression()"]:

    model_prod = eval(model_name).fit(X.iloc[id_nonprod, :], y.iloc[id_nonprod])
    mae_prod = mean_absolute_error(y.iloc[id_prod], model_prod.predict(X.iloc[id_prod, :]))
    
    # 3. for each of the 12 validation methods...
    for validation_method in validation_methods:

      # 4. for each fold...
      for enum, (id_train, id_test) in enumerate(eval(validation_method2function[validation_method])):

        if len(id_train) == 0 or len(id_test) == 0:
          continue

        # fit model on train set
        model_val = eval(model_name).fit(X.iloc[id_train, :], y.iloc[id_train])
        
        # compute MAE on training and on test set
        mae_train = mean_absolute_error(y.iloc[id_train], model_val.predict(X.iloc[id_train, :]))
        mae_test = mean_absolute_error(y.iloc[id_test], model_val.predict(X.iloc[id_test, :]))

        # save results
        results.loc[results_row, "dataset_id"] = dataset_id
        results.loc[results_row, "validation_method"] = validation_method
        results.loc[results_row, "model_name"] = model_name
        results.loc[results_row, "enum"] = enum
        results.loc[results_row, "len_X"] = len(X)
        results.loc[results_row, "pct_len_train"] = len(id_train) / len(X)
        results.loc[results_row, "pct_len_test"] = len(id_test) / len(X)
        results.loc[results_row, "pct_len_prod"] = len(id_prod) / len(X)
        results.loc[results_row, "mae_train"] = mae_train
        results.loc[results_row, "mae_test"] = mae_test
        results.loc[results_row, "mae_prod"] = mae_prod

        results_row += 1

# average over different iterations of the same validation method (e.g. cross-validation)
results = results.groupby(["dataset_id", "validation_method", "model_name"]).mean().reset_index()

# the (relative) difference between test and prod metric is the measure of how good the validation method is
results["test_prod_change"] = (results["mae_test"] / results["mae_prod"] - 1).abs()

# Results

In [8]:
# pivot dataset and method, using test prod change as a kpi

test_prod_change = pd.crosstab(results["dataset_id"], results["validation_method"], results["test_prod_change"], aggfunc=np.mean)
test_prod_change.columns.name = None

In [9]:
# show test-prod deviance

test_prod_change.loc[:, validation_methods].head(10).style.set_precision(2).highlight_min(axis=1)

,holdout,inv_holdout,rep_holdout,cv,cv_mod,cv_bl,cv_hvbl,preq_bls,preq_sld_bls,preq_bls_gap,preq_slide,preq_grow
dataset_id,,,,,,,,,,,,
020,0.02,0.00,0.09,0.10,0.04,0.09,0.09,0.10,0.02,0.04,0.17,0.16
091,0.05,0.09,0.01,0.04,0.09,0.05,0.05,0.05,0.09,0.07,0.03,0.03
092,0.06,0.16,0.06,0.09,0.14,0.09,0.09,0.09,0.12,0.09,0.04,0.04
093,0.09,0.13,0.15,0.16,0.12,0.16,0.16,0.16,0.14,0.11,0.01,0.00
159,0.18,0.15,0.16,0.03,0.04,0.02,0.02,0.03,0.02,0.03,0.22,0.21
190,0.20,0.03,0.24,0.18,0.16,0.18,0.18,0.21,0.20,0.21,0.13,0.13
191,0.76,0.97,0.77,0.89,0.88,0.89,0.89,0.87,0.87,0.84,0.76,0.76
205,0.25,0.46,0.56,0.28,0.34,0.44,0.44,0.85,0.60,2.14,0.17,0.17
210,0.14,0.50,0.11,0.14,0.28,0.15,0.15,0.10,0.12,0.13,0.13,0.12


In [10]:
# how many times a method is the best one?

test_prod_change.idxmin(axis=1).value_counts()

inv_holdout     13
rep_holdout      7
preq_sld_bls     7
preq_grow        7
cv_mod           6
preq_bls_gap     5
preq_bls         3
cv_bl            2
preq_slide       2
holdout          2
cv_hvbl          2
cv               2
dtype: int64

In [11]:
# how correlated is each method with the others, on average?

test_prod_change.corr().mask(np.diag([True]*test_prod_change.shape[1])).mean().round(2).sort_values()

inv_holdout     0.86
preq_bls_gap    0.86
rep_holdout     0.88
cv_mod          0.91
preq_bls        0.93
preq_grow       0.93
preq_slide      0.93
holdout         0.94
cv              0.95
preq_sld_bls    0.95
cv_bl           0.96
cv_hvbl         0.96
dtype: float64

In [12]:
# how worse is the second best method, on average?

test_prod_change.apply(lambda row: row.sort_values().head(2).iloc[1]/row.min()-1, axis=1).groupby(test_prod_change.idxmin(axis=1)).mean().round(2).sort_values(ascending=False)

cv_mod          9.73
inv_holdout     4.19
preq_sld_bls    4.17
preq_bls        3.12
preq_bls_gap    1.89
rep_holdout     1.18
holdout         0.93
cv_bl           0.36
preq_grow       0.21
cv_hvbl         0.17
cv              0.03
preq_slide      0.02
dtype: float64